In [1]:
from anime_downloader import get_anime_class

In [2]:
masterani = get_anime_class("masteranime")

In [8]:
x = masterani('https://www.masterani.me/anime/info/53-hunter-x-hunter-2011')

In [26]:
y = x.verify_url('https://www.masterani.me/watch/53-hunter-x-hunter-2011/1')

In [27]:
y

True

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#
# direct and api method from:
#   streamondemand: https://github.com/streamondemand/plugin.video.streamondemand/blob/master/servers/openload.py
#
# Edited by:    Twoure
# Edit Date:    05/17/17

from operator import ixor

USER_AGENT = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
RE_NORM = Regex(r'(https?://\w+\.\w+)/\w+/([^/]+)(/.+)?')

def link_from_api(fid):
    l = 'OTdiMjMyNmQ3ZGI4MWYwZg__'
    k = 'QVFGTzNRSlE_'
    burl = 'aHR0cHM6Ly9hcGkub3BlbmxvYWQuY28vMS9maWxlLw__'
    api1 = 'e31kbHRpY2tldD9maWxlPXt9JmxvZ2luPXt9JmtleT17fQ__'
    api2 = 'e31kbD9maWxlPXt9JnRpY2tldD17fQ__'

    try:
        data = JSON.ObjectFromURL(D(api1).format(D(burl), fid, D(l), D(k)), cacheTime=CACHE_1HOUR*5)
    except:
        Log.Exception(u"* <openload.link_from_api> - error: cannot handle first api link >>>")
        return False

    if (data["status"] == 200):
        t = data["result"]["ticket"]
        try:
            data = JSON.ObjectFromURL(D(api2).format(D(burl), fid, t), cacheTime=CACHE_1HOUR*5)
            return (data['result']['url']).replace("https", "http")
        except:
            Log.Exception(u"* <openload.link_from_api> - error: cannot handle 2nd api link >>>")
    Log.Error("* <openload.link_from_api> - error: failed to retrieve video stream")
    return False

def direct_decode(encode, numeros, ops):
    Log("* <openload.direct_decode> - Trying direct method 05/17/17.")
    text_decode = ""
    try:
        mult = int(ops[0]) * int(ops[1])
        rango1 = encode[:mult]
        decode1 = []
        for i in range(0, len(rango1), 8):
            decode1.append(int(rango1[i:i+8], 16))
        rango1 = encode[mult:]
        j = 0
        i = 0
        while i < len(rango1):
            index1 = 64
            value1 = 0
            value2 = 0
            value3 = 0
            while True:
                if (i + 1) >= len(rango1):
                    index1 = 143
                value3 = int(rango1[i:i+2], 16)
                i += 2
                data = value3 & 63
                value2 += data << value1
                value1 += 6
                if value3 < index1:
                    break

            value4 = value2 ^ decode1[j % (mult/8)]
            for n in numeros:
                if not n.isdigit():
                    n = int(n, 16)
                value4 = ixor(value4, int(n))
            value5 = index1 * 2 + 127
            for h in range(4):
                valorfinal = (value4 >> 8 * h) & (value5)
                valorfinal = chr(valorfinal)
                if valorfinal != "%":
                    text_decode += valorfinal
            j += 1
        return unicode(text_decode)
    except:
        Log.Exception("* <openload.direct_decode> - error[1]: cannot directly decode with method 05/17/17 >>>")
    return False

def decode_hiddenUrl(hurls, page):
    r = Regex(r'(?s)\(\'\_\'\)\;(.+?)ﾟωﾟﾉ= /｀ｍ´）ﾉ').search(page)
    if not r:
        Log.Error("* <openload.decode_hiddenUrl> - error: cannot find jscode in page, with method 03/29/17 >>>")
        return False
    jscode = r.group(1)
    rn = Regex(r'_[A-f0-9]+x[A-f0-9]+\s*(?:=|\^)\s*([0-9]{4,}|0x[A-f0-9]{4,})').findall(jscode)
    rops = Regex(r'\(0x(\d),0x(\d)\);').search(jscode)
    ops = rops.groups() if rops else list()

    for (hid, k) in hurls:
        if " " in k:
            continue
        dec = direct_decode(k, rn, ops)
        if dec:
            return dec
    return False

def OpenloadStreamFromURL(url, http_headers=None):
    if not http_headers:
        http_headers = {'User-Agnet': USER_AGENT, 'Referer': url}

    base = RE_NORM.search(url)
    eurl = base.group(1) + '/embed/' + base.group(2) + (base.group(3) if base.group(3) else '')

    try:
        page = HTTP.Request(eurl, encoding=('utf-8'), headers=http_headers, cacheTime=CACHE_1MINUTE).content
    except UnicodeDecodeError, ude:
        Log.Warn(u"* Warning: Content removed by Openload for '{0}'".format(eurl))
        Log(str(ude))
        return False
    except:
        Log(u"* Error handling '{0}' >>>".format(eurl))
        Log.Exception(u"* Error: Cannot Open/Decode Openload page >>>")
        return False

    html = HTML.ElementFromString(page)
    hiddenUrls = html.xpath('//span[@id]')
    if hiddenUrls:
        hurl = decode_hiddenUrl([(h.get('id'), h.text) for h in hiddenUrls], page)
        if hurl:
            return u'https://openload.co/stream/{0}?mime=true'.format(hurl)
        else:
            Log.Error(u'* Cannot directly decode hiddenUrl. Trying api method.')
            # Fallback to api method, although at peak times it will not work
            return link_from_api(base.group(2))
    else:
        Log.Warn(u'* No hiddenUrl to decode.')
    return False

SyntaxError: invalid syntax (<ipython-input-1-91887fb305dc>, line 110)